<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/Multi_head_Self_Attention_for_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import tensorflow as tf

In [24]:
#멀티 헤드 어텐션
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, embedding_dim, num_heads=8):
    super(MultiHeadAttention, self).__init__()
    self.embedding_dim=embedding_dim #d_model
    self.num_heads=num_heads

    assert embedding_dim%num_heads==0

    self.projection_dim=embedding_dim//num_heads
    self.query_dense=tf.keras.layers.Dense(embedding_dim)
    self.key_dense=tf.keras.layers.Dense(embedding_dim)
    self.value_dense=tf.keras.layers.Dense(embedding_dim)
    self.dense=tf.keras.layers.Dense(embedding_dim)

  def scaled_dot_product_attention(self, query, key, value):
    matmul_qk=tf.matmul(query, key, transpose_b=True) #qeury와 key의 전치행렬의 곱 = 점수
    depth=tf.cast(tf.shape(key)[-1], tf.float32) #key의 shape의 마지막에 위치한 값(벡터의 차원)을 float로 바꾼다. -> dk
    logits=matmul_qk/tf.math.sqrt(depth) #루트 dk로 나눈다
    attention_weights=tf.nn.softmax(logits, axis=-1) #softmax함수를 지나 점수를 비율로 바꾼다.
    output=tf.matmul(attention_weights, value) #value와 곱해주고 값을 얻는다.
    return output, attention_weights

  def split_heads(self, x, batch_size):
    x=tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim)) #batch_size(크기)와 num_heads(헤드의 수)를 통해 헤드들을 나눈다.
    return tf.transpose(x, perm=[0,2,1,3])#두번째 차원과 세번째 차원을 바꿔서 리턴한다.

  def call(self, inputs):
    #x.shape=(batch_size, seq_len, embedding_dim)
    batch_size=tf.shape(inputs)[0] #inputs의 shape의 첫번째 행에 있는 batch_size를 사용한다.

    #(batch_size, seq_len, embedding_dim)
    query=self.query_dense(inputs)
    key=self.key_dense(inputs)
    value=self.value_dense(inputs)
    #각각 inputs와 층을 연결한다.

    #(batch_size, num_heads, seq_len, projection_dim)
    qeury=self.split_heads(query, batch_size)
    key=self.split_heads(key, batch_size)
    value=self.split_heads(value, batch_size)
    #각각 헤드들을 분리하여 num_heads의 차원이 하나가 추가되고, embedding_dim은 num_heads로 나누어져 procjection_dim(dk)가 된다.

    scaled_attention, _ = self.scaled_dot_product_attention(query, key, value) #행렬곱과 점수, softmax를 통해 확률을 구하고 값을 구한다.
    #(batch_size, seq_len, num_heads, projection_dim)
    scaled_attention=tf.transpose(scaled_attention, perm=[0,2,1,3])#2번째 차원과 3번째 차원의 축을 바꿔준다.

    #(batch_size, seq_len, embedding_dim)
    concat_attention=tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim)) #reshape으로 차원을 축소한다.
    outputs=self.dense(concat_attention) #모두 이은 어텐션을 출력층에 놓는다.
    return outputs

In [25]:
class TransformerBlock(tf.keras.layers.Layer):
  def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
    super(TransformerBlock, self).__init__()
    self.att = MultiHeadAttention(embedding_dim, num_heads)
    self.ffn=tf.keras.Sequential([tf.keras.layers.Dense(dff, activation='relu'),tf.keras.layers.Dense(embedding_dim),]) #맨 마지막에 한 번더 가중치를 곱해주는 코드(dff로 곱했다가 다시 d_model로 곱하여 원래의 shape으로 복귀)
    self.layernorm1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.dropout1=tf.keras.layers.Dropout(rate)
    self.dropout2=tf.keras.layers.Dropout(rate)

  def call(self, inputs, training):
    attn_output=self.att(inputs) #첫번째 서브층 : 멀티 헤드 어텐션
    attn_output=self.dropout1(attn_output, training=training)
    out1=self.layernorm1(inputs+attn_output) #Add+Norm 잔차 처리 및 층 정규화
    ffn_output=self.ffn(out1) #두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
    ffn_output=self.dropout2(ffn_output, training=training)
    return self.layernorm2(out1+ffn_output)#Add+Norm 잔차처리 및 층 정규화

In [26]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
  def __init__(self, max_len, vocab_size, embedding_dim):
    super(TokenAndPositionEmbedding, self).__init__()
    self.token_emb=tf.keras.layers.Embedding(vocab_size, embedding_dim) #임베딩
    self.pos_emb=tf.keras.layers.Embedding(max_len, embedding_dim) #포지셔닝

  def call(self, x):
    max_len=tf.shape(x)[-1]
    positions=tf.range(start=0, limit=max_len, delta=1)
    positions=self.pos_emb(positions)
    x=self.token_emb(x)
    return x+positions #임베딩과 포지셔닝이 된 상태로 리턴

In [27]:
vocab_size = 20000  # 빈도수 상위 2만개의 단어만 사용
max_len = 200  # 문장의 최대 길이

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수 : {}'.format(len(X_train)))
print('테스트용 리뷰 개수 : {}'.format(len(X_test)))


훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


In [28]:
X_train=tf.keras.preprocessing.sequence.pad_sequences(X_train)
X_test=tf.keras.preprocessing.sequence.pad_sequences(X_test)

In [29]:
embedding_dim = 32  # 각 단어의 임베딩 벡터의 차원
num_heads = 2  # 어텐션 헤드의 수
dff = 32  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(2, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


ValueError: ignored

In [ ]:
model.compile